In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

### Đọc dữ liệu

In [2]:
file_path = './ProcessedData.csv'
data = pd.read_csv(file_path)

### Xử lý dữ liệu

In [3]:
# Remove unnecessary columns and prepare features and target
# Dropping 'Unnamed: 0', 'timestamp', and 'Weather Description' for now
columns_to_drop = ['Unnamed: 0', 'timestamp', 'Weather Description']
air = ['so2','no2','pm10','pm2_5','o3','co']
weather = ['Temprature (Kelvin)','Feels like (Kelvin)','Pressure (mb)',
           'Humidity (%)','Temperature Amplitude','Wind speed (m/s)','Wind degree',
           'Wind gust (m/s)','Clouds all (%)','Rain 1h (mm)']
features = data.drop(columns=columns_to_drop + ['aqi'] + air)  # Exclude target column 'aqi' and air
target = data[air]

### Xây dựng hàm loss

#### Dictionary để tham chiếu AQI dựa trên các chỉ số thành phần không khí

In [4]:
aqi_thresholds = {
    'so2': [
        (0, 20), (20, 80), (80, 250), (250, 350), (350, float('inf'))
    ],
    'no2': [
        (0, 40), (40, 70), (70, 150), (150, 200), (200, float('inf'))
    ],
    'pm10': [
        (0, 20), (20, 50), (50, 100), (100, 200), (200, float('inf'))
    ],
    'pm2_5': [
        (0, 10), (10, 25), (25, 50), (50, 75), (75, float('inf'))
    ],
    'o3': [
        (0, 60), (60, 100), (100, 140), (140, 180), (180, float('inf'))
    ],
    'co': [
        (0, 4400), (4400, 9400), (9400, 12400), (12400, 15400), (15400, float('inf'))
    ]
}

#### Trả về danh sách các chất gây ra ô nhiễm dựa trên các thành phần không khí trong 1 dòng

In [5]:
def identify_pollutants(row, thresholds, pollutant_names):
    """
    Xác định mức AQI của một mảng dữ liệu và trả về các chất ô nhiễm với mức AQI tương ứng.

    Parameters:
    - row: list hoặc numpy array chứa các giá trị của chất ô nhiễm.
    - thresholds: từ điển ngưỡng AQI.
    - pollutant_names: danh sách tên các chất ô nhiễm tương ứng với row.

    Returns:
    - Danh sách các chất ô nhiễm cùng với mức AQI của chúng.
    """
    aqi_details = []  # Danh sách lưu trữ chi tiết các chất ô nhiễm và mức AQI
    for i, pollutant in enumerate(pollutant_names):
        value = row[i]  # Giá trị của chất ô nhiễm tại vị trí i
        for index, (lower, upper) in enumerate(thresholds[pollutant], start=1):
            if lower <= value < upper:
                if index >= 4:
                    aqi_details.append(pollutant)  # Lưu tên chất ô nhiễm và mức AQI
                break
    return aqi_details  # Trả về danh sách các chất ô nhiễm và mức AQI

#### Xác định ô nhiễm gây ra bởi các nhóm chất nào (bụi mịn (PM2.5, PM10)/ Khí độc (CO, SO2, NO)/ Ozone tầng mặt đất (O3))
Các chất ô nhiễm cùng nhóm có các nguyên nhân gây ra và biện pháp phòng tránh gần tương tự nhau

In [6]:
def group_pollutant(pollutants):
    groups = [['pm2_5', 'pm10'],['co','so2','no2'], ['o3']]
    idx_array = []
    for item in pollutants:
        for index in range(len(groups)):
            if item in groups[index] and index not in idx_array:
                idx_array.append(index)
                
    return idx_array

#### Hàm loss
Trả ra kết quả đúng/sai. Nếu mẫu không khí dự đoán và mẫu không khí thực tế có cùng các nhóm chất ô nhiễm được xem là đúng. Khác nhau sẽ là sai

In [7]:
def loss_function(y_pred, y_test):
    count_diff = 0
    for i in range(len(y_test)):
        aqi_predict = identify_pollutants(y_pred.loc[i], aqi_thresholds, air)
        aqi_test = identify_pollutants(y_test.iloc[i].values, aqi_thresholds, air)
        if group_pollutant(aqi_predict) != group_pollutant(aqi_test):
            count_diff += 1
    
    return count_diff / len(y_test)

#### Chia tập
Chia tập dữ liệu theo tỉ lệ 80/10/10: Mô hình có đủ dữ kiện để học, tinh chỉnh tham số và kiểm tra xem mô hình có bị overfitting/underfitting hay không

In [8]:
# Chia dữ liệu thành tập train (80%), validation (10%) và test (10%)
# Tách tập test (10%)
X_train_valid, X_test, y_train_valid, y_test = train_test_split(features, target, test_size=0.1, random_state=42)

# Tách tập train và valid (1/9 của train_valid)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.1/0.9, random_state=42)

### Mô hình cơ sở: Hồi quy tuyến tính đơn biến

Vì mô hình hồi quy tuyến tính không có tham số điều chỉnh do đó dùng tập dữ liệu train/test đã được chia ở trên để huấn luyện mô hình

In [9]:
y_pred_lg = []

for col in y_train.columns:
    model = LinearRegression()
    model.fit(X_train, y_train[col])
    y_pred_lg.append(model.predict(X_test))

# Chuyển về dataframe
y_pred_lg = pd.DataFrame(y_pred_lg).T
y_pred_lg.columns = y_train.columns

In [10]:
y_pred_lg

,so2,no2,pm10,pm2_5,o3,co
0,50.684416,44.340690,73.784391,50.632736,31.903188,1295.719836
1,26.432663,30.373145,47.828544,29.053158,1.024751,918.481214
2,36.029755,35.083525,65.972743,47.193143,10.207127,1252.355067
3,54.057710,52.729967,104.287249,68.323767,25.215685,1673.105170
4,42.121971,36.125891,62.158936,45.839362,13.103738,1215.222753
...,...,...,...,...,...,...
790,26.249933,26.790209,14.616847,12.705371,16.104098,576.945508
791,47.202143,43.752227,71.069961,46.219408,45.065024,1118.374568
792,34.370276,33.276423,38.433383,27.150634,13.997871,946.111655
793,30.323394,30.063691,20.080068,15.602415,26.212929,665.017587


##### Tính loss của mô hình

In [11]:
loss = loss_function(y_pred_lg, y_test)
print('Loss: ', loss)

Loss:  0.13836477987421383


#### Mô hình cơ sở: Cây quyết định

Mô hình cây quyết định sử dụng thêm tập dữ liệu valid để tinh chỉnh siêu tham số

In [12]:
validation_loss = []
max_depth = 31

for depth in range(1, max_depth):
    # Huấn luyện
    model = DecisionTreeRegressor(max_depth=depth, random_state=42)
    model.fit(X_train, y_train)
    # Dự đoán trên valid
    y_pred_valid = model.predict(X_valid)
    # Chuyển sang df
    y_pred_valid_df = pd.DataFrame(y_pred_valid, columns=y_train.columns)
    # Tính loss
    loss = loss_function(y_pred_valid_df, y_valid)
    # Lưu kết quả
    validation_loss.append((depth, loss))

best_depth, best_loss = min(validation_loss, key = lambda x: x[1])
print(f"Best max_depth: {best_depth}, Validation Loss: {best_loss}")

Best max_depth: 14, Validation Loss: 0.1320754716981132


##### Huấn luyện mô hình cuối cùng

In [13]:
final_model = DecisionTreeRegressor(max_depth=best_depth, random_state=42)
final_model.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=14, random_state=42)

In [14]:
y_pred_test = final_model.predict(X_test)
y_pred_test_df = pd.DataFrame(y_pred_test, columns=y_train.columns)
y_pred_test_df

,so2,no2,pm10,pm2_5,o3,co
0,43.630000,26.820000,44.645000,32.190000,17.350000,1034.740000
1,27.535000,34.790000,49.325000,37.865000,14.795000,1034.740000
2,32.406250,26.167969,65.195781,51.908125,0.514062,1353.919531
3,46.704074,51.243333,74.562222,50.303704,3.120741,1477.558889
4,52.450000,34.275000,31.215000,20.145000,18.775000,841.140000
...,...,...,...,...,...,...
790,21.847273,25.111818,16.548182,10.181818,18.742727,400.239091
791,65.800000,70.600000,45.700000,36.540000,41.130000,1188.280000
792,36.409048,37.927619,34.198571,21.629048,11.052381,786.463333
793,23.398333,25.018611,22.929167,18.092500,12.363889,716.156389


In [15]:
# loss
loss = loss_function(y_pred_test_df, y_test)
print('Loss: ', loss)

Loss:  0.15220125786163521


### Mô hình hồi quy đa biến

#### Chia tập
Mô hình hồi quy đa biến không sử dụng rừng ngẫu nhiên làm mô hình cơ sở, việc chia train/valid được thực hiện tự động bởi hàm GridSearchCV dựa trên tập dữ liệu đầu vào là X_train, y_train đã chia ở trên. Grid search sử dụng k-fold cross-validation để tinh chỉnh siêu tham số tự động

### Phần này mô hình chạy tương đối lâu

In [16]:
# Định nghĩa mô hình RandomForestRegressor
rf = RandomForestRegressor()

# Định nghĩa mô hình MultiOutputRegressor với RandomForestRegressor
model = MultiOutputRegressor(rf)

# Tạo một GridSearchCV với các tham số cần tinh chỉnh
param_grid = {
    'estimator__n_estimators': [50, 100, 200],
    'estimator__max_depth': [10, 20, None],
    'estimator__min_samples_split': [2, 5, 10],
    'estimator__min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1)

# Huấn luyện mô hình và tìm tham số tốt nhất
grid_search.fit(X_train, y_train)

# In ra các tham số tối ưu
print("Best parameters found: ", grid_search.best_params_)

# Dự đoán với mô hình đã được tối ưu
predictions = grid_search.predict(X_test)

Best parameters found:  {'estimator__max_depth': 20, 'estimator__min_samples_leaf': 2, 'estimator__min_samples_split': 2, 'estimator__n_estimators': 200}


In [17]:
pred_df = pd.DataFrame(predictions, columns=y_train.columns)
pred_df

,so2,no2,pm10,pm2_5,o3,co
0,65.060264,59.448862,57.463505,46.052380,19.811606,1243.774436
1,38.913668,35.667446,39.737693,30.881989,3.389377,799.003837
2,30.159542,27.603567,58.964453,48.401910,0.177802,1259.411296
3,49.714553,46.558927,60.897849,48.915929,11.685559,1369.503625
4,40.088422,32.804428,53.589479,44.534680,2.191228,1204.967177
...,...,...,...,...,...,...
790,22.563794,29.091425,16.881366,11.129595,20.031714,454.940337
791,80.941646,67.823583,89.001669,73.828793,33.616755,1292.936059
792,34.728734,34.003968,33.564321,22.770804,1.942830,847.437489
793,25.074664,24.791399,22.097382,19.472705,22.565729,728.534662


In [18]:
loss = loss_function(pred_df, y_test)
print('Loss: ', loss)

Loss:  0.1119496855345912
